In [15]:
"""
Detect a face in webcam video and check if mouth is open above 3CM.
"""
import face_recognition
import cv2
from datetime import datetime
import math

def get_lip_height(lip):
    sum=0
    for i in [2,3,4]:
        # distance between two near points up and down
        distance = math.sqrt( (lip[i][0] - lip[12-i][0])**2 +
                              (lip[i][1] - lip[12-i][1])**2   )
        sum += distance
    return sum / 3

def get_mouth_height(top_lip, bottom_lip):
    sum=0
    for i in [8,9,10]:
        # distance between two near points up and down
        distance = math.sqrt( (top_lip[i][0] - bottom_lip[18-i][0])**2 + 
                              (top_lip[i][1] - bottom_lip[18-i][1])**2   )
        sum += distance
        print(sum)
    return sum / 3

# ratio 1.5 = mouth is opened 1.5 times of the lip_hight (My lip hight aproximately = 2cm[top+bottom 
def check_mouth_open(top_lip, bottom_lip):
    top_lip_height =    get_lip_height(top_lip)
    bottom_lip_height = get_lip_height(bottom_lip)
    mouth_height =      get_mouth_height(top_lip, bottom_lip)

    # if mouth is open more than lip height * ratio, return true.
    ratio = 0.5
    if mouth_height > min(top_lip_height, bottom_lip_height) * ratio * 3 :
        return True
    else:
        return False

def is_mouth_open(face_landmarks):
    top_lip = face_landmarks['top_lip']
    bottom_lip = face_landmarks['bottom_lip']

    top_lip_height = get_lip_height(top_lip)
    bottom_lip_height = get_lip_height(bottom_lip)
    mouth_height = get_mouth_height(top_lip, bottom_lip)
    
    # if mouth is open more than lip height * ratio, return true.
    ratio = 0.5
    #print('top_lip_height: %.2f, bottom_lip_height: %.2f, mouth_height: %.2f, min*ratio: %.2f' 
    #      % (top_lip_height,bottom_lip_height,mouth_height, min(top_lip_height, bottom_lip_height) * ratio))
          
    if mouth_height > min(top_lip_height, bottom_lip_height) * ratio *3 :
        return True
    else:
        return False


# Get a reference to webcam #0 (the default one)
width = 640
height = 480
video_capture = cv2.VideoCapture(0, cv2.CAP_DSHOW)
video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, width)
video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
video_capture.set(cv2.CAP_PROP_FPS, 30)
video_capture.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))


# Define the codec and create VideoWriter object to save video to local
fourcc = cv2.VideoWriter_fourcc(*'XVID') # codec
#cv2.VideoWriter( filename, fourcc, fps, frameSize )
out = cv2.VideoWriter('output.avi', fourcc, 1, (640, 480)) 

# Load a sample picture and learn how to recognize it.
ujwal_image = face_recognition.load_image_file("C:\\Users\\Magic\\Videos\\Screenshot 2022-08-26 02-44-23.png") # replace ujwal.jpg with your own image !!
ujwal_face_encoding = face_recognition.face_encodings(ujwal_image)[0]

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    # Find all the faces and face enqcodings in the frame of video
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)
    face_landmarks_list = face_recognition.face_landmarks(frame)

    # Loop through each face in this frame of video
    for (top, right, bottom, left), face_encoding, face_landmarks in zip(face_locations, face_encodings, face_landmarks_list):

        #  See if the face is a match for the known face(s)
        match = face_recognition.compare_faces([ujwal_face_encoding], face_encoding)

        name = "Unknown"
        if match[0]:
            name = "Ujwal"

#        # Draw a box around the face
#        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
#        cv2.rectangle(frame, (left, bottom), (right, bottom + 35), (0, 0, 255), cv2.FILLED)
#        font = cv2.FONT_HERSHEY_DUPLEX
#        cv2.putText(frame, name, (left + 6, bottom + 25), font, 1.0, (255, 255, 255), 1)


        # Display text for mouth open / close
        ret_mouth_open = is_mouth_open(face_landmarks)
        if ret_mouth_open is True:
            text = 'Mouth is open above 3CM'
        else:
            text = ''
        cv2.putText(frame, text, (left - 60, top - 50), cv2.FONT_HERSHEY_DUPLEX, 1.0, (255, 255, 255), 1)
        #Draw circle edge of lip
        top_lip = face_landmarks['top_lip']
        right_edge_lip = top_lip[0]
        left_edge_lip = top_lip[6]
        #print(right_edge_lip)
        #print(left_edge_lip)
        cv2.circle(frame, (right_edge_lip), 3, (0,0,255), 3), cv2.circle(frame, (left_edge_lip), 3, (0,0,255), 3)
        
    # Display the resulting image
    cv2.imshow('Video', frame)
    out.write(frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

1.0
3.0
4.414213562373095
2.0
4.0
5.414213562373095
2.0
4.0
5.414213562373095
6.324555320336759
12.649110640673518
17.748130154266303
16.1245154965971
33.1539018625235
50.1832882284499
1.4142135623730951
2.8284271247461903
4.242640687119286
2.0
3.414213562373095
4.414213562373095
1.4142135623730951
2.8284271247461903
5.06449510224598
4.123105625617661
8.246211251235321
12.7183472062349
17.029386365926403
34.058772731852805
50.089992273734204
2.23606797749979
4.47213595499958
6.708203932499369
2.0
4.23606797749979
5.23606797749979
1.0
2.0
3.414213562373095
1.0
3.0
6.0
8.246211251235321
17.465755708528206
25.71196695976353
1.4142135623730951
2.8284271247461903
5.06449510224598
1.0
3.0
5.0
1.0
3.0
5.23606797749979
14.035668847618199
28.071337695236398
42.213473318967345
23.021728866442675
46.04345773288535
69.06518659932803
1.4142135623730951
2.8284271247461903
5.06449510224598
2.0
4.0
6.0
2.0
4.0
6.0
8.246211251235321
16.492422502470642
24.55468025076919
21.095023109728988
43.18574514410